# Example code of GPT API
Colab 환경에서 GPT 모델 API를 사용해 손글씨를 인식해보는 예제입니다.

## Colab 환경 설정
예제를 실행시키기 위해 python package들을 설치합니다. 예제로 사용할 이미지들도 다운로드 받습니다.

In [1]:
# Local에서 Run하는 경우 False로 변경
using_colab = True

In [2]:
if using_colab:
    !wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/ocr/requirements.txt
    !pip install -r requirements.txt

## Import dependency

In [16]:
import openai
import json

## OpenAI GPT API 설정
OpenAI Platform의 계정에 연결된 API 키를 사용합니다. 그리고 어떤 모델을 사용할 지 설정합니다. 이 예제에서는 `gpt-3.5-turbo`를 사용합니다.

In [3]:
# openai API 키 인증
openai.api_key = "<OPENAI_API_KEY>"

# 모델 - GPT 3.5 Turbo 선택
model = "gpt-3.5-turbo"

## Inference Model

### 프롬프트 작성하여 GPT API에 요청해보기

In [11]:
# 프롬프트 작성하기
prompt = "Describe ChatGPT in 40 words or less."

In [12]:
# 메시지 설정하기
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

### GPT API 호출하기

In [13]:
# GPT API 호출하기
response = openai.ChatCompletion.create(model=model, messages=messages)

In [14]:
response

<OpenAIObject chat.completion id=chatcmpl-7w4yApzxFO08abzi0wmoEyNk954uO at 0x111c74d10> JSON: {
  "id": "chatcmpl-7w4yApzxFO08abzi0wmoEyNk954uO",
  "object": "chat.completion",
  "created": 1694076482,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "ChatGPT is an AI-powered language model designed to engage in natural language conversations. It can understand and generate human-like responses on a wide range of topics, making it useful for various applications like customer support, drafting emails, and more."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 28,
    "completion_tokens": 48,
    "total_tokens": 76
  }
}

In [15]:
response["choices"][0]["message"]["content"]

'ChatGPT is an AI-powered language model designed to engage in natural language conversations. It can understand and generate human-like responses on a wide range of topics, making it useful for various applications like customer support, drafting emails, and more.'

### Function calling

In [25]:
def get_current_weather(location: str):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "20",
        "unit": "Celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [18]:
function_signature = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
        },
        "required": ["location"],
    },
}

In [19]:
prompt = "What's the weather like in Seoul?"
messages = [{"role": "user", "content": prompt}]

response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=[function_signature],
)
response_message = response["choices"][0]["message"]

In [20]:
response_message

<OpenAIObject at 0x1190a1540> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}

In [21]:
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = get_current_weather(
        location=function_args["location"],
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )

    second_response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )

In [22]:
second_response["choices"][0]["message"]["content"]

'The current weather in Seoul is sunny and windy with a temperature of 20 degrees Celsius.'